# Interrupt Control
* Provides support for syscalls (trap), device event handling (int)
    * Fault handling
* **register** points to a table in memory
* An **IDT** (interrupt descriptor table) entry consists of
    * handler - pointer addr to a piece of code that gets executed
    * access permissions
        * eg. iret instruction
        * depending on which interrupt entry you hit determines which level of privilege you gain
        * only care about supervisor+user modes
    * event type - eg. interrupt (dev), trap (proc), fault (proc)
* Every interrupt has a number associated to it (0-255)
    * internal interrupts - eg. trap
    * external interrupts - eg. from devices

## Expanded view on Instruction Execution
1. Fetch
2. EIP is incremented to addr's next instruction
3. Execute instruction, result is reflected in flags, local, and maybe EIP
4. Read/Write memory
5. Check for interrupt (save and change PC + memory descriptors on interrupt)
    * PC at this moment is pointing to the next instruction
    * PC gets pushed onto the stack
    * When we enter the interrupt, the previous PC is the top item on the stack

# Memory Management (MMU)
* Supported through GDT register
* Assists paging, memory protections, caching
* Register pointing to a table in memory
* Table comprises entries that directly/indirectly contain
    * page tables
    * permissions
    * valid ranges
    * memory access info

## What happens on a memory reference?
* selector (current value in GDT register) to index the MMU (GDT) table
* Processor checks that the access if valid, which means
    * the addr is valid
    * the processor has sufficient privilege to access the addr
    * page is in memory
    * if anything fails, fault (trap/exception) is generated
  

# Interrupts

1. **HW:** CPU is notified that an interrupt $I$ occuured
    * save some state
        * eg. previous CPU permission, PC
        * we want this to be the minimal amount of state to be saved because this is atomic
    * CPU switch to supervisor mode
    * CPU indexes the IDT to find the interrupt hander
    * Jump to handler code
2. **SW:** Handler services the interrupt
    * Save some more CPU state
    * Load "old" kernel state, enter kernel
    * Perform service/function
    * Save current kernel state
    * Restore CPU state (in handler scope)
        * We could put new values into registers as well here (as our response)
        * We could switch to a different process state (this would allow us to "swap" processes)
3. **HW:** Perform $iret$
    * Switch to user mode
    * Restores state from first step (cpu scope)

# What can we do with this stuff?
* Guarantee integrity between processes (memory protection)
* Allocate CPU fairly through preemption (timer interrupt)
* Provide a uniform abstraction of devices to all processes (all devices accessed via syscalls)
